In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 71.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import faiss
import numpy as np
import os
import pickle
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from sentence_transformers import SentenceTransformer
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/final_processed_complaints.csv')
df.head()

,Product,cleaned_narrative
0,Credit card,a xxxx xxxx card was opened under my name by a...
1,Credit card,dear cfpb i have a secured credit card with ci...
2,Credit card,i have a citi rewards cards the credit balance...
3,Credit card,bi am writing to dispute the following charges...
4,Credit card,although the account had been deemed closed i ...


chenking the text

In [ ]:
docs = [Document(page_content=t, metadata={"product": df.iloc[i]["Product"]}) for i, t in enumerate(df['cleaned_narrative'])]

# Create chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)

print(f"Total chunks created: {len(chunks)}")
print("Sample chunk:", chunks[0].page_content)


Buffered data was truncated after reaching the output size limit.

 Generate Embeddings

In [ ]:
model = SentenceTransformer("all-MiniLM-L6-v2", device='cuda')
embeddings = model.encode(
    [doc.page_content for doc in chunks],
    batch_size=64,
    show_progress_bar=True,
    convert_to_numpy=True
)

Batches:   0%|          | 0/4034 [00:00<?, ?it/s]

 FAISS Vector Store

In [ ]:
# Create and save FAISS index
dimension = embeddings[0].shape[0]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

faiss.write_index(index, "/content/drive/MyDrive/complaint_index.faiss")

In [ ]:
with open("/content/drive/MyDrive/chunk_metadata.pkl", "wb") as f:
    pickle.dump([doc.metadata for doc in chunks], f)

with open("/content/drive/MyDrive/chunk_texts.pkl", "wb") as f:
    pickle.dump([doc.page_content for doc in chunks], f)